In [1]:
import time
import pyodbc
import pandas as pd
import threading
import ccxt

# Initialize the KuCoin and Binance clients
kucoin = ccxt.kucoin()
binance = ccxt.binance()
bitget = ccxt.bitget()  # Initialize Bitget client


# List of coins to fetch data for
coins = ["sxp", "chess", "blz", "joe", "perl", "ach", "gmt", "xrp", "akro", "zil", "cfx", "adx", "chz", "bel", "alpaca", "elf", "epx", "pros", "t", "dar", "agix", "mob", "id", "trx", "key", "tru", "amb", "magic", "lina", "lever"]

def get_db_connection():
    connection = pyodbc.connect(
        driver='{SQL Server}',
        server='A2NWPLSK14SQL-v06.shr.prod.iad2.secureserver.net',
        database='db_ran',
        uid='dbahsantrade',
        pwd='Pak@1947'
    )
    return connection

# Create tables for each coin
def create_tables():
    connection = get_db_connection()
    cursor = connection.cursor()
    for coin in coins:
        cursor.execute(f"""
            IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'{coin}usdt') AND type in (N'U'))
            BEGIN
                CREATE TABLE {coin}usdt (
                    id INT IDENTITY(1,1) PRIMARY KEY,
                    timestamp DATETIME DEFAULT GETDATE(),
                    price DECIMAL(18, 8),
                    volume DECIMAL(18, 8)
                )
            END
        """)
    connection.commit()
    connection.close()

# Function to check if coin is supported on an exchange
def is_coin_supported(exchange, coin):
    markets = exchange.load_markets()
    return f"{coin.upper()}/USDT" in markets

# Fetch and store recent trades
def fetch_and_store_recent_trades(exchange, coin):
    last_trade_timestamp = None
    while True:
        try:
            connection = get_db_connection()
            cursor = connection.cursor()
            trades = exchange.fetch_trades(f'{coin.upper()}/USDT')
            for trade in trades:
                trade_timestamp = pd.to_datetime(trade['timestamp'], unit='ms')
                if last_trade_timestamp is None or trade_timestamp > last_trade_timestamp:
                    cursor.execute(f"INSERT INTO {coin}usdt (timestamp, price, volume) VALUES (?, ?, ?)",
                                   (trade_timestamp, trade['price'], trade['amount']))
                    last_trade_timestamp = trade_timestamp
            connection.commit()
        except Exception as e:
            print(f'Error fetching and storing trades for {coin} from {exchange.name}: {e}')
            connection.rollback()
            # Wait for 30 seconds before retrying
            time.sleep(30)
            continue
        finally:
            connection.close()
        time.sleep(5)

# Function to delete data older than 7 Days
def delete_old_data(coin):
    while True:
        try:
            connection = get_db_connection()
            cursor = connection.cursor()
            
            # Deleting in batches of 1000 rows to avoid large transactions
            rows_deleted = 1  # Initialize with dummy value to enter the loop
            while rows_deleted > 0:
                cursor.execute(f"""
                    DELETE TOP (1000)
                    FROM {coin}usdt
                    WHERE timestamp < DATEADD(DAY, -7, GETDATE())
                """)
                rows_deleted = cursor.rowcount
                connection.commit()
        except Exception as e:
            print(f'Error deleting old data for {coin}: {e}')
            if connection:
                connection.rollback()
        finally:
            if connection:
                connection.close()
        
        # Sleep for a certain period before running the deletion again. This example uses 6 hours.
        time.sleep(21600)  # Sleep for 6 hours

# Start threads for each coin and exchange
def start_threads():
    create_tables()
    for coin in coins:
        # Check for coin support in Binance
        if is_coin_supported(binance, coin):
            threading.Thread(target=fetch_and_store_recent_trades, args=(binance, coin)).start()
            threading.Thread(target=delete_old_data, args=(coin,)).start()

        # Check for coin support in KuCoin
        if is_coin_supported(kucoin, coin):
            threading.Thread(target=fetch_and_store_recent_trades, args=(kucoin, coin)).start()
            threading.Thread(target=delete_old_data, args=(coin,)).start()

        # Check for coin support in Bitget
        if is_coin_supported(bitget, coin):  # Add this block for Bitget
            threading.Thread(target=fetch_and_store_recent_trades, args=(bitget, coin)).start()
            threading.Thread(target=delete_old_data, args=(coin,)).start()
# Start threads
start_threads()

# Keep the script running
while True:
    time.sleep(30)
